In [12]:
#import libraries needed for this proces
import requests
from datetime import datetime
import numpy
import time
import json
from pathlib import Path
import pandas as pd
import sys

In [2]:
#function to create an API request based on URL and  parameters
def get_request(url, parameters=None):
    print("Your request is being processed.")
    response = requests.get(url=url, params=parameters)
    if response:
        return response.json()
        
    else:
        print("Request unsuccessful, retrying in 5 seconds.")
        time.sleep(5)
        return get_request(url, parameters)
    time.sleep(1)

In [3]:
#using the function to get all data id's which include appid and app name 
url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
parameters = {"request": all}
data_ids = get_request(url, parameters=parameters)
print("Your request has been processed successfully, call upon 'data_ids' to preview the full list of IDs and names of all currently available products on Steam as of ", datetime.now().strftime("%d/%m/%Y %H:%M:%S"), ".")

Your request is being processed.
Your request has been processed successfully, call upon 'data_ids' to preview the full list of IDs and names of all currently available products on Steam as of  20/03/2022 15:48:12 .


In [21]:
#Code to collect all app id's in one library
app_ids = []
print("Creating a list of all IDs currently available on Steam")
for item in data_ids['applist']['apps']:
    app_ids.append(item['appid'])

print("List created sucessfully, the total number of IDs available on Steam is:", len(app_ids),".")
print("Call upon 'app_ids' to preview the full list of extracted IDs.")

print("Saving the Steam ID list in the 'data' folder as 'steam_id_list.csv'.")
numpy.savetxt("../gen/steam_id_list.csv", app_ids,delimiter =", ", fmt ='% s')
print("File 'steam_id_list.csv' has been saved successfully!")


Creating a list of all IDs currently available on Steam
List created sucessfully, the total number of IDs available on Steam is: 138483 .
Call upon 'app_ids' to preview the full list of extracted IDs.
Saving the Steam ID list in the 'data' folder as 'steam_id_list.csv'.
File 'steam_id_list.csv' has been saved successfully!


In [9]:
## collecting the data with a for loop that checks if the response was a succes = True. and than takes the data attribute 
# feedback: seperate the raw data collection, and then do the parsing  
# add timestamp 
# add logic that the id's arent run multiple times 

amount_app_ids = 10
counter = 1 
dataframe_raw = pd.DataFrame()
dataframe_categories = pd.DataFrame()
dataframe_package_groups =  pd.DataFrame()
dataframe_genres =  pd.DataFrame()
dataframe_screenshots =  pd.DataFrame()
dataframe_movies =  pd.DataFrame()
dataframe_achievements = pd.DataFrame()

for i in app_ids: 
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {i}}
    app_data = get_request(url, parameters=parameters)
    id = str(i)
    print("Processing data pertaining to Steam ID:"+id)
    response = app_data[id]
    succes = response["success"]
    if (succes): 
        data_json = response["data"]
        result = pd.json_normalize(data_json)
        dataframe_raw = pd.concat([dataframe_raw, result],ignore_index= True)    
        try:
            categories = pd.json_normalize(data_json,record_path = ['categories'], meta = ["steam_appid"], errors="ignore")
            dataframe_categories = pd.concat([dataframe_categories,categories], ignore_index= True)
        except:
            pass
        try:
            package_groups = pd.json_normalize(data_json,record_path = ['package_groups'], meta = ["steam_appid"], errors="ignore")
            dataframe_package_groups = pd.concat([dataframe_package_groups,package_groups], ignore_index= True)
        except:
            pass
        try:
            genres = pd.json_normalize(data_json,record_path = ['genres'], meta = ["steam_appid"], errors="ignore")
            dataframe_genres = pd.concat([dataframe_genres,genres], ignore_index= True)
        except:
            pass
        try:
            screenshots = pd.json_normalize(data_json,record_path = ['screenshots'], meta = ["steam_appid"], errors="ignore")
            dataframe_screenshots = pd.concat([dataframe_screenshots,screenshots], ignore_index= True)
        except:
            pass
        try:
            movies = pd.json_normalize(data_json,record_path = ['movies'], meta = ["steam_appid"], errors="ignore")
            dataframe_movies = pd.concat([dataframe_movies,movies], ignore_index= True)
        except:
            pass
        try:
            achievements = pd.json_normalize(data_json,record_path = ['achievements.highlighted'], meta = ["steam_appid"], errors="ignore")
            dataframe_achievements = pd.concat([dataframe_achievements,achievements], ignore_index= True)
        except:
            pass
        counter = counter + 1 
    if (counter > amount_app_ids):
        break


print("Saving all data about the Steam ID's in the 'gen' folder as 'steam_data_collection'.")
numpy.savetxt("../gen/full_steam_data.json", dataframe_raw, delimiter=", ", fmt = '% s')
print("File 'full_steam_data.json' has been saved successfully!")




Your request is being processed.
Processing data pertaining to Steam ID:216938
Your request is being processed.
Processing data pertaining to Steam ID:660010
Your request is being processed.
Processing data pertaining to Steam ID:660130
Your request is being processed.
Processing data pertaining to Steam ID:1118314
Your request is being processed.
Processing data pertaining to Steam ID:1275822
Your request is being processed.
Processing data pertaining to Steam ID:1343832
Your request is being processed.
Processing data pertaining to Steam ID:1828741
Your request is being processed.
Processing data pertaining to Steam ID:662172
Your request is being processed.
Processing data pertaining to Steam ID:1360782
Your request is being processed.
Processing data pertaining to Steam ID:1820332
Your request is being processed.
Processing data pertaining to Steam ID:1815690
Your request is being processed.
Processing data pertaining to Steam ID:1815720
Your request is being processed.
Processing 

In [13]:
## collecting the data with a for loop that checks if the response was a succes = True. and than takes the data attribute 
# feedback: seperate the raw data collection, and then do the parsing  
# add timestamp 
# add logic that the id's arent run multiple times 

import json

def get_data_from_api(amount_app_ids = 100):
    counter = 1 
    dataframe_raw = pd.DataFrame()
    
    for i in app_ids: 
        url = "http://store.steampowered.com/api/appdetails/"
        parameters = {"appids": {i}}
        app_data = get_request(url, parameters=parameters)
        id = str(i)
        print("Processing data pertaining to Steam ID:"+id)
        response = app_data[id]
        succes = response["success"]
        if (succes): 
            data_json = response["data"]
            # enrich it
            data_json['collection_details'] = {'created_by': 'our_scraper',
                                                'created_at': 12345678}
            f = open('raw_data.json','a',encoding='utf-8')
            f.write(json.dumps(data_json)+'\n')
            f.close()
            #result = pd.json_normalize(data_json)
            #dataframe_raw = pd.concat([dataframe_raw, result],ignore_index= True)    
            counter = counter + 1 
        if (counter > amount_app_ids):
            break
        #return dataframe_raw

get_data_from_api(10) 
#dataframe_raw


Your request is being processed.
Processing data pertaining to Steam ID:216938
Your request is being processed.
Processing data pertaining to Steam ID:660010
Your request is being processed.
Processing data pertaining to Steam ID:660130
Your request is being processed.
Processing data pertaining to Steam ID:1118314
Your request is being processed.
Processing data pertaining to Steam ID:1275822
Your request is being processed.
Processing data pertaining to Steam ID:1343832
Your request is being processed.
Processing data pertaining to Steam ID:1828741
Your request is being processed.
Processing data pertaining to Steam ID:662172
Your request is being processed.
Processing data pertaining to Steam ID:1360782
Your request is being processed.
Processing data pertaining to Steam ID:1820332
Your request is being processed.
Processing data pertaining to Steam ID:1815690
Your request is being processed.
Processing data pertaining to Steam ID:1815720
Your request is being processed.
Processing 

In [8]:
# split up in seperate functions: get raw data, parse data, 

dataframe_raw
#dataframe_categories
#dataframe_package_groups 
#dataframe_genres 
#dataframe_screenshots
#dataframe_movies 
#dataframe_achievements 

,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,...,price_overview.final_formatted,mac_requirements,linux_requirements,legal_notice,fullgame.appid,fullgame.name,metacritic.score,controller_support,achievements.total,achievements.highlighted
0,game,Employee A,1815690,0,False,"<img src=""https://cdn.akamai.steamstatic.com/s...","<img src=""https://cdn.akamai.steamstatic.com/s...",A pixel art point-and-click adventure set in t...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,game,Split Tactics,1815720,0,False,Split Tactics is a casual game about choices. ...,Split Tactics is a casual game about choices. ...,A turn-based LAN multiplayer card game where p...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,...,"3,29€",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,game,BallRoll,1815730,0,False,Experience metaphorical descent into the land ...,Experience metaphorical descent into the land ...,Experience metaphorical descent into the land ...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,...,"0,79€",[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,game,The Healer,1815740,0,False,"<h2 class=""bb_tag"">THOUSANDS OF POSSIBILITIES!...","<h2 class=""bb_tag"">THOUSANDS OF POSSIBILITIES!...",Can you get zero kills? It's a different exper...,English<strong>*</strong><br><strong>*</strong...,https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,demo,Expeditions: Rome Demo,1815770,0,True,"<h2 class=""bb_tag"">The future of Rome is in yo...","<h2 class=""bb_tag"">The future of Rome is in yo...",Determine the destiny of Rome as you conquer f...,"English<strong>*</strong>, French, German<stro...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,"© 2021 THQ Nordic AB, Sweden. Developed by Log...",987840,Expeditions: Rome,80.0,NaN,NaN,NaN
5,game,How to Fool a Liar King Remastered,1815800,0,False,"The cute fantasy visual novel, &quot;How to Fo...","The cute fantasy visual novel, &quot;How to Fo...","The cute fantasy visual novel, &quot;How to Fo...","English, French, German, Japanese<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,full,NaN,NaN
6,game,How to Sing to Open Your Heart Remastered,1815820,0,False,"The cute fantasy visual novel, &quot;How to Si...","The cute fantasy visual novel, &quot;How to Si...","The cute fantasy visual novel, &quot;How to Si...","English, French, German, Japanese<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,full,NaN,NaN
7,game,Maze Art: Black,1815860,0,False,Maze Art: Black - is a minimalist game of simp...,Maze Art: Black - is a minimalist game of simp...,Maze Art: Black - is a minimalist game of simp...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,"1,59€",[],[],NaN,NaN,NaN,NaN,NaN,100.0,"[{'name': 'Level 1', 'path': 'https://cdn.akam..."
8,game,Maze Art: Blue,1815880,0,False,Maze Art: Blue - is a minimalist game of simpl...,Maze Art: Blue - is a minimalist game of simpl...,Maze Art: Blue - is a minimalist game of simpl...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,"1,59€",[],[],NaN,NaN,NaN,NaN,NaN,100.0,"[{'name': 'Level 1', 'path': 'https://cdn.akam..."
9,game,Lights,1815890,0,False,Lights is an endless runner in which you need ...,Lights is an endless runner in which you need ...,Lights is an endless runner in which you need ...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,...,"1,--€",[],[],NaN,NaN,NaN,NaN,NaN,62.0,"[{'name': 'A', 'path': 'https://cdn.akamai.ste..."


In [37]:
#write the dataframes to an xlsx file
dataframe_raw.to_excel("D:\\1. Studie\\Master files\\dPrep-oDCM\\Git repo\\Steam-API\\steam-API\\data\\dataraw_df.xlsx")
dataframe_categories.to_excel("D:\\1. Studie\\Master files\\dPrep-oDCM\\Git repo\\Steam-API\\steam-API\\data\\datacategories_df.xlsx")

In [12]:
### example to create a dictionary that checks if the id is already in the dictionary and then if not in dictionary get data 

"""
details = {}

for i in app_ids:
  id = str(i)
  if id not in details:
     // get data

     if(success):
       // put data in details
       details[id] = data

"""     

'\ndetails = {}\n\nfor i in app_ids:\n  id = str(i)\n  if id not in details:\n     // get data\n\n     if(success):\n       // put data in details\n       details[id] = data\n\n'